# INTRO TO ARTIFICIAL INTELLIGENCE
# FINAL PROJECT
## NEURAL NETWORK FOR DIGIT RECOGNITION TRAINED FROM MNIST DATASET

### By Carlo D. Pastoral - N26087048
[YouTube Video Presentation ](https://youtu.be/AUh1xQiWESk)

# Packages

To build my neuralnetwork, I used the following Packages:

* Numpy - Used for manipulating and computing large multi-dimensional arrays and matrices. 
* Tensorflow - A library for symbolic math used for neural networks.
* Terflow Datasets - An API provided by Tensorflow for the ease of accessing a wide range of open source and free datasets. 


In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


# Dataset

* Using the **load** function of the **tensorflow_datasets**, I downloaded the **MNIST** dataset directly from tensorflow. 
* This function also allows me to simultaeneously split the dataset into train and test sets.
* Aside from splitting the data, I am also able to shuffle it.
* **tfds.load** function, also ships the dataset info that is very useful in knowing the content of the dataset, its reference, and most importantly the dataset features that can be used for data preparation.

In [19]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_info

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.1,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

# Data Preprocessing

* In this part I applied a normalization for each image in the dataset. 
* Normalizing is mainly used for making all the data in the same scale. 
* Normalizing the data also helps reducing the computing requirements.

* **Caching the dataset** - the **cache** fucntion of tfds is useful in loading the dataset during training as it is making it faster and significantly reducing memory consumption. 



In [ ]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.cache()
ds_train = ds_train.batch(128)
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

# Neural Network Model

* Here I designed a very simple FCNN that has only 3 layers. Input, hidden layer, and Output.

* **Flatten** - as an input layer transforms the 2D image into a vector. 
* **Dense**  - As a hidden layer that produces 128 feature maps activated by relu. 
* **Dense**  - As a output layer that produces 62 probabilities activated by softmax. 


In [76]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(62, activation='softmax')
])
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=5,
    validation_data=ds_test,
)

Epoch 1/5
469/469 [==============================] - 4s 9ms/step - loss: 0.4443 - accuracy: 0.8831 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1952 - accuracy: 0.9461 - val_loss: 0.1540 - val_accuracy: 0.9552
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1422 - accuracy: 0.9597 - val_loss: 0.1270 - val_accuracy: 0.9623
Epoch 4/5
469/469 [==============================] - 2s 5ms/step - loss: 0.1117 - accuracy: 0.9683 - val_loss: 0.1113 - val_accuracy: 0.9663
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0911 - accuracy: 0.9749 - val_loss: 0.1004 - val_accuracy: 0.9689


In [79]:
model.save('MNIST_.31%_Carlo_Pastoral.h5')

# Results

* For a very simple FCNN, it gives me a ~96% accuracy. 
* To further test the model, here I reloaded the mnist testset without shuffling. 
* I used 10 images from selected indexes to made a prediction

In [78]:
test_loss, test_acc = model.evaluate(ds_test, verbose=2)
print('\nTest accuracy:', test_acc)

image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='test', 
    batch_size=-1, 
    as_supervised=True,
))

predictions = model.predict(image) 
print(np.argmax(predictions[50:60],axis=1))
print(label[50:60])

79/79 - 0s - loss: 0.1004 - accuracy: 0.9689

Test accuracy: 0.9689
[0 7 9 9 9 6 5 8 8 6]
[0 7 9 9 9 6 5 8 8 6]
